### Base Model

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "gpt2"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Input text
prompt = "Explain why the sky is blue"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate without LoRA
with torch.no_grad():
    base_out = base_model.generate(**inputs, max_new_tokens=20)

print("=== Base model output ===")
print(tokenizer.decode(base_out[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== Base model output ===
Explain why the sky is blue.

The sky is blue.

The sky is blue.

The sky is


### LoRA Finetuned Model

In [4]:
from peft import PeftModel

# Load base model again
model_with_lora = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Attach LoRA adapter
model_with_lora = PeftModel.from_pretrained(model_with_lora, "outputs/lora_adapter")
model_with_lora.to(device)

# Generate with LoRA
with torch.no_grad():
    lora_out = model_with_lora.generate(**inputs, max_new_tokens=20)

print("\n=== Model with LoRA output ===")
print(tokenizer.decode(lora_out[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



=== Model with LoRA output ===
Explain why the sky is blue and why it's blue.

The sky is blue because it is blue. It is blue


In [8]:
prompts=[
    "Explain why the sky is blue",
    "List the steps for making a peanut butter and jelly sandwich",
    "Write a short story about a robot who learns to paint.",
    "Compare cats and dogs as pets in a few sentences."
]

for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate without LoRA
    with torch.no_grad():
        base_out = base_model.generate(**inputs, max_new_tokens=50)
        lora_out = model_with_lora.generate(**inputs, max_new_tokens=50)

    print(f"Prompt: {prompt}\n")
    print("=== Base model output ===")
    print(tokenizer.decode(base_out[0], skip_special_tokens=True))
    print("\n=== Model with LoRA output ===")
    print(tokenizer.decode(lora_out[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Explain why the sky is blue

=== Base model output ===
Explain why the sky is blue.

The sky is blue.

The sky is blue.

The sky is blue.

The sky is blue.

The sky is blue.

The sky is blue.

The sky is blue.

=== Model with LoRA output ===
Explain why the sky is blue and why it's blue.

The sky is blue because it is blue. It is blue because it is blue because it is blue because it is blue because it is blue because it is blue because it is blue because it is blue because it


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: List the steps for making a peanut butter and jelly sandwich

=== Base model output ===
List the steps for making a peanut butter and jelly sandwich.

Step 1:

1. In a large bowl, combine the butter, sugar, and salt.

2. Add the eggs, milk, and vanilla.

3. Add the flour and mix well.



=== Model with LoRA output ===
List the steps for making a peanut butter and jelly sandwich.

Step 1: Preheat oven to 350 degrees F.

Step 2: In a large bowl, whisk together the butter, sugar, and vanilla.

Step 3: Add the eggs, vanilla, and salt.




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Write a short story about a robot who learns to paint.

=== Base model output ===
Write a short story about a robot who learns to paint.

The story is about a robot who learns to paint.

The story is about a robot who learns to paint.

The story is about a robot who learns to paint.

The story is about a robot who learns to

=== Model with LoRA output ===
Write a short story about a robot who learns to paint.

"I'm a robot," says the robot, "and I'm learning to paint."

"I'm learning to paint," says the robot, "and I'm learning to paint."

"I'm learning to paint,"


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Compare cats and dogs as pets in a few sentences.

=== Base model output ===
Compare cats and dogs as pets in a few sentences.

"I'm not going to say that I'm a cat lover, but I'm not going to say that I'm a dog lover either," he said. "I'm not going to say that I'm a dog lover either."


=== Model with LoRA output ===
Compare cats and dogs as pets in a few sentences.

"I'm not sure if it's a good idea to have cats or dogs in your home, but I think it's a good idea to have them in your home," she said.

"I think it's important to have
